In [2]:
import gym

from stable_baselines.common.policies import MlpPolicy, MlpLstmPolicy, MlpLnLstmPolicy, CnnPolicy, CnnLstmPolicy, CnnLnLstmPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import A2C

from gym.utils import seeding
from stable_baselines.common.env_checker import check_env
import numpy as np

from ads_utils import load_data, plot, Environment
from tqdm import tqdm

from random import randint

In [3]:
import wandb

In [4]:
sweep_config = {
  "name": "a2c sweep",
  "method": "grid",
  "parameters": {
      "learning_rate": {
            "values": [0.001, 0.0001, 0.00001]
        },
      "n_steps": {
          "values": [10, 50, 70, 100]  
        },
      "n_ticks": {
          "values": [5, 20, 50, 100]
      },
      "momentum": {
        "values": [0, 0.1, 0.5, 0.9]  
      }
    }
}

In [5]:
sweep_id = wandb.sweep(sweep_config, entity="ads", project="a2c-sweep-all-params-val-data2")

Create sweep with ID: cmh1del2
Sweep URL: https://wandb.ai/ads/a2c-sweep-all-params-val-data2/sweeps/cmh1del2


In [6]:
training_data = load_data([i for i in range(1, 12 + 1)])['close'].to_list()
val_data = load_data([i for i in range(13, 18 + 1)])['close'].to_list()

In [7]:
def create_training_env(ticks):
    INITIAL_BALANCE = 10_000
    # sample training data
#     start = randint(0, len(training_data) - 10000 - 1) 
#     sample = training_data[start: start + 10000]
    return Environment(training_data, balance=INITIAL_BALANCE, past_ticks=ticks)

In [8]:
def create_validation_env(ticks):
    INITIAL_BALANCE = 10_000
    return Environment(val_data, balance=INITIAL_BALANCE, past_ticks=ticks)

In [9]:
def train():
    run = wandb.init()
    print("config:", dict(run.config))
    
    N_EPOCH = 50
    env = create_training_env(run.config.n_ticks)
    env.reset()
    model = A2C('MlpPolicy', env, verbose=0, 
                learning_rate=run.config.learning_rate, 
                n_steps=run.config.n_steps,
                momentum=run.config.momentum)
    
    for i in range(N_EPOCH):
        print(f'Epoch {i}')
        model.learn(total_timesteps=10000)

    val_env = create_validation_env(run.config.n_ticks)
    state = val_env.reset(rand_start=False)
    
    for i in range(len(val_data)):
        action, _ = model.predict(state)

        price, portfolio_value = val_env.get_data()

        state, reward, done, _ = val_env.step(action)
        wandb.log({'portfolio_value': portfolio_value, 'reward': reward})
        if done:
            break

In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: zzyhc0cf with config:
wandb: 	learning_rate: 0.001
wandb: 	momentum: 0
wandb: 	n_steps: 10
wandb: 	n_ticks: 5
wandb: Currently logged in as: ads (use `wandb login --relogin` to force relogin)
